# Pràctica 2: Creació de la visualització i lliurament del projecte

Autor: Ernest Panareda Roig

In [1]:
# Importem llibreries
import sqlite3
import pandas as pd

In [2]:
# Llegim la base de dades que conté les taules
con = sqlite3.connect("database.sqlite")

In [3]:
# Estudiem les taules
tables = ["Country",
          "League",
          "Match",
          "Player",
          "Player_Attributes",
          "Team",
          "Team_Attributes"]

t_col = dict()

for t in tables:
    df = pd.read_sql_query("SELECT * FROM {}".format(t), con)
    t_col[t] = [len(df), len(df.columns), list(df.columns)]
    print("La taula {} té {} files i {} columnes anomenades:\n{}\n"
          .format(t, len(df), len(df.columns), list(df.columns)))

La taula Country té 11 files i 2 columnes anomenades:
['id', 'name']

La taula League té 11 files i 3 columnes anomenades:
['id', 'country_id', 'name']

La taula Match té 25979 files i 115 columnes anomenades:
['id', 'country_id', 'league_id', 'season', 'stage', 'date', 'match_api_id', 'home_team_api_id', 'away_team_api_id', 'home_team_goal', 'away_team_goal', 'home_player_X1', 'home_player_X2', 'home_player_X3', 'home_player_X4', 'home_player_X5', 'home_player_X6', 'home_player_X7', 'home_player_X8', 'home_player_X9', 'home_player_X10', 'home_player_X11', 'away_player_X1', 'away_player_X2', 'away_player_X3', 'away_player_X4', 'away_player_X5', 'away_player_X6', 'away_player_X7', 'away_player_X8', 'away_player_X9', 'away_player_X10', 'away_player_X11', 'home_player_Y1', 'home_player_Y2', 'home_player_Y3', 'home_player_Y4', 'home_player_Y5', 'home_player_Y6', 'home_player_Y7', 'home_player_Y8', 'home_player_Y9', 'home_player_Y10', 'home_player_Y11', 'away_player_Y1', 'away_player_Y2', '

In [4]:
# Obtenim les dades de les taules amb les que es treballaran.
matchs_t = pd.read_sql_query("SELECT * FROM {}".format("Match"), con)
country_t = pd.read_sql_query("SELECT * FROM {}".format("Country"), con)
league_t = pd.read_sql_query("SELECT * FROM {}".format("League"), con)
team_t = pd.read_sql_query("SELECT * FROM {}".format("Team"), con)

# Definim el diccionari amb les dades dels països i lligues
cl_dic = dict()
for c in zip(country_t['id'],
             country_t['name']):
    for l in zip(league_t['country_id'],
                 league_t['name']):
        if c[0] == l[0]:
            cl_dic[c[0]] = {"country": c[1], "league": l[1]}

#Definim diccionari amb dades dels equips            
team_dic = dict()
for t in zip(team_t['team_api_id'],
             team_t['team_long_name']):
    team_dic[t[0]] = t[1]

        
#print(cl_dic)

#Definim diccionari amb totes les dades a estudiar agrupades en el format d'interès
match_season = dict()
count = 0
for m in zip(matchs_t['id'], #0
             matchs_t['country_id'], #1
             matchs_t['season'], #2
             matchs_t['home_team_api_id'], #3
             matchs_t['away_team_api_id'], #4
             matchs_t['home_team_goal'], #5
             matchs_t['away_team_goal']): #6
    
    # Validar la temporada
    if m[2] in match_season.keys():
        #Validar la lliga
        if cl_dic[m[1]]["league"] in match_season[m[2]].keys():
            #Validar equip local
            if team_dic[m[3]] in match_season[m[2]][cl_dic[m[1]]["league"]].keys():
                match_season[m[2]][cl_dic[m[1]]["league"]][team_dic[m[3]]]["socres"] += m[5]
                match_season[m[2]][cl_dic[m[1]]["league"]][team_dic[m[3]]]["scores_home"] += m[5]                
                match_season[m[2]][cl_dic[m[1]]["league"]][team_dic[m[3]]]["socres_rec"] += m[6]
                match_season[m[2]][cl_dic[m[1]]["league"]][team_dic[m[3]]]["scores_rec_home"] += m[6]
                
                if int(m[5])>int(m[6]):
                    match_season[m[2]][cl_dic[m[1]]["league"]][team_dic[m[3]]]["wins"] += 1
                    match_season[m[2]][cl_dic[m[1]]["league"]][team_dic[m[3]]]["wins_home"] += 1
                elif int(m[5])==int(m[6]):
                    match_season[m[2]][cl_dic[m[1]]["league"]][team_dic[m[3]]]["draws"] += 1
                    match_season[m[2]][cl_dic[m[1]]["league"]][team_dic[m[3]]]["draws_home"] += 1
                else:
                    match_season[m[2]][cl_dic[m[1]]["league"]][team_dic[m[3]]]["loses"] += 1
                    match_season[m[2]][cl_dic[m[1]]["league"]][team_dic[m[3]]]["loses_home"] += 1
                
            else:
                match_season[m[2]][cl_dic[m[1]]["league"]][team_dic[m[3]]] = {"socres": m[5],
                                     "scores_home": m[5],
                                     "scores_away": 0,
                                     "socres_rec": m[6],
                                     "scores_rec_home": m[6],
                                     "scores_rec_away": 0,
                                     "wins": 1 if int(m[5])>int(m[6]) else 0,
                                     "wins_home": 1 if int(m[5])>int(m[6]) else 0,
                                     "wins_away": 0,
                                     "draws": 1 if int(m[5])==int(m[6]) else 0,
                                     "draws_home": 1 if int(m[5])==int(m[6]) else 0,
                                     "draws_away": 0,
                                     "loses": 1 if int(m[5])<int(m[6]) else 0,
                                     "loses_home": 1 if int(m[5])<int(m[6]) else 0,
                                     "loses_away": 0}
                
            #Validar equip visitant
            if team_dic[m[4]] in match_season[m[2]][cl_dic[m[1]]["league"]].keys():
                match_season[m[2]][cl_dic[m[1]]["league"]][team_dic[m[4]]]["socres"] += m[6]
                match_season[m[2]][cl_dic[m[1]]["league"]][team_dic[m[4]]]["scores_away"] += m[6]
                match_season[m[2]][cl_dic[m[1]]["league"]][team_dic[m[4]]]["socres_rec"] += m[5]
                match_season[m[2]][cl_dic[m[1]]["league"]][team_dic[m[4]]]["scores_rec_away"] += m[5]
                
                if int(m[5])<int(m[6]):
                    match_season[m[2]][cl_dic[m[1]]["league"]][team_dic[m[4]]]["wins"] += 1
                    match_season[m[2]][cl_dic[m[1]]["league"]][team_dic[m[4]]]["wins_away"] += 1
                elif int(m[5])==int(m[6]):
                    match_season[m[2]][cl_dic[m[1]]["league"]][team_dic[m[4]]]["draws"] += 1
                    match_season[m[2]][cl_dic[m[1]]["league"]][team_dic[m[4]]]["draws_away"] += 1
                else:
                    match_season[m[2]][cl_dic[m[1]]["league"]][team_dic[m[4]]]["loses"] += 1
                    match_season[m[2]][cl_dic[m[1]]["league"]][team_dic[m[4]]]["loses_away"] += 1
                
                
            else:
                match_season[m[2]][cl_dic[m[1]]["league"]][team_dic[m[4]]] = {"socres": m[6],
                                     "scores_home": 0,
                                     "scores_away": m[6],
                                     "socres_rec": m[5],
                                     "scores_rec_home": 0,
                                     "scores_rec_away": m[5],
                                     "wins": 1 if int(m[5])<int(m[6]) else 0,
                                     "wins_home": 0,
                                     "wins_away": 1 if int(m[5])<int(m[6]) else 0,
                                     "draws": 1 if int(m[5])==int(m[6]) else 0,
                                     "draws_home": 0,
                                     "draws_away": 1 if int(m[5])==int(m[6]) else 0,
                                     "loses": 1 if int(m[5])>int(m[6]) else 0,
                                     "loses_home": 0,
                                     "loses_away": 1 if int(m[5])>int(m[6]) else 0}
                
        else:
            match_season[m[2]][cl_dic[m[1]]["league"]] = {team_dic[m[3]]: {"socres": m[5],
                                     "scores_home": m[5],
                                     "scores_away": 0,
                                     "socres_rec": m[6],
                                     "scores_rec_home": m[6],
                                     "scores_rec_away": 0,
                                     "wins": 1 if int(m[5])>int(m[6]) else 0,
                                     "wins_home": 1 if int(m[5])>int(m[6]) else 0,
                                     "wins_away": 0,
                                     "draws": 1 if int(m[5])==int(m[6]) else 0,
                                     "draws_home": 1 if int(m[5])==int(m[6]) else 0,
                                     "draws_away": 0,
                                     "loses": 1 if int(m[5])<int(m[6]) else 0,
                                     "loses_home": 1 if int(m[5])<int(m[6]) else 0,
                                     "loses_away": 0},
                     team_dic[m[4]]: {"socres": m[6],
                                     "scores_home": 0,
                                     "scores_away": m[6],
                                     "socres_rec": m[5],
                                     "scores_rec_home": 0,
                                     "scores_rec_away": m[5],
                                     "wins": 1 if int(m[5])<int(m[6]) else 0,
                                     "wins_home": 0,
                                     "wins_away": 1 if int(m[5])<int(m[6]) else 0,
                                     "draws": 1 if int(m[5])==int(m[6]) else 0,
                                     "draws_home": 0,
                                     "draws_away": 1 if int(m[5])==int(m[6]) else 0,
                                     "loses": 1 if int(m[5])>int(m[6]) else 0,
                                     "loses_home": 0,
                                     "loses_away": 1 if int(m[5])>int(m[6]) else 0}}
        
    else:
        match_season[m[2]] = {cl_dic[m[1]]["league"]:
                              {team_dic[m[3]]: {"socres": m[5],
                                     "scores_home": m[5],
                                     "scores_away": 0,
                                     "socres_rec": m[6],
                                     "scores_rec_home": m[6],
                                     "scores_rec_away": 0,
                                     "wins": 1 if int(m[5])>int(m[6]) else 0,
                                     "wins_home": 1 if int(m[5])>int(m[6]) else 0,
                                     "wins_away": 0,
                                     "draws": 1 if int(m[5])==int(m[6]) else 0,
                                     "draws_home": 1 if int(m[5])==int(m[6]) else 0,
                                     "draws_away": 0,
                                     "loses": 1 if int(m[5])<int(m[6]) else 0,
                                     "loses_home": 1 if int(m[5])<int(m[6]) else 0,
                                     "loses_away": 0},
                               team_dic[m[4]]: {"socres": m[6],
                                     "scores_home": 0,
                                     "scores_away": m[6],
                                     "socres_rec": m[5],
                                     "scores_rec_home": 0,
                                     "scores_rec_away": m[5],
                                     "wins": 1 if int(m[5])<int(m[6]) else 0,
                                     "wins_home": 0,
                                     "wins_away": 1 if int(m[5])<int(m[6]) else 0,
                                     "draws": 1 if int(m[5])==int(m[6]) else 0,
                                     "draws_home": 0,
                                     "draws_away": 1 if int(m[5])==int(m[6]) else 0,
                                     "loses": 1 if int(m[5])>int(m[6]) else 0,
                                     "loses_home": 0,
                                     "loses_away": 1 if int(m[5])>int(m[6]) else 0}}}
    # Punt de control    
    '''count+=1
    if count>500:
        break'''
        
#print(match_season)

In [5]:
#Es prenen les dades per poder treballar amb el diccionari
seasons = list(match_season.keys())
leagues = list(match_season[seasons[0]].keys())
teams = []
for s in seasons:
    for l in leagues:
        for t in list(match_season[s][l].keys()):
            teams.append(t)
            
teams = set(teams)
            
print("El nombre total d'equips que juguen alguna temporada en les lligues europees és de {}."
     .format(len(teams)))

El nombre total d'equips que juguen alguna temporada en les lligues europees és de 296.


In [6]:
#Obtenir gols totals, a casa i fora.
score_home = 0
score_away = 0

for s in seasons:
    for l in leagues:
        for t in list(match_season[s][l].values()):
            score_home+=t["scores_home"]
            score_away+=t["scores_away"]
            
score_tt = score_home+score_away
            
print("El número de gols totals a casa és de {}, i fora de casa de {}. El total de gols és de {}."
     .format(score_home, score_away, score_tt))
            

#Obtenir mitjana de gols per partit

print("La mitjana de cols per partit és de {}."
      .format(round(score_tt/25979, 2))) #Número partits = número de files a la taula "Match".

El número de gols totals a casa és de 40127, i fora de casa de 30160. El total de gols és de 70287.
La mitjana de cols per partit és de 2.71.


In [7]:
# Configurem el diccionari per obtenir les dades a representar: gols per lligues i temporades.
data_goals_tt = dict()
index_goals_tt = list()

for s in seasons:
    data_goals_tt[s] = []
    for l in leagues:
        index_goals_tt.append(l)
        goals_count = 0
        for t in list(match_season[s][l].values()):
            goals_count += t["socres"]
        data_goals_tt[s].append(goals_count)
        

# Definim el dataset amb les dades.
df_goals_tt = pd.DataFrame(data_goals_tt, index = index_goals_tt[:11])
df_goals_tt

,2008/2009,2009/2010,2010/2011,2011/2012,2012/2013,2013/2014,2014/2015,2015/2016
Belgium Jupiler League,855,565,635,691,703,30,668,694
England Premier League,942,1053,1063,1066,1063,1052,975,1026
France Ligue 1,858,916,890,956,967,933,947,960
Germany 1. Bundesliga,894,866,894,875,898,967,843,866
Italy Serie A,988,992,955,925,1003,1035,1018,979
Netherlands Eredivisie,870,892,987,997,964,978,942,912
Poland Ekstraklasa,524,532,578,527,598,634,628,635
Portugal Liga ZON Sagres,552,601,584,634,667,569,763,831
Scotland Premier League,548,585,584,601,623,626,587,650
Spain LIGA BBVA,1101,1031,1042,1050,1091,1045,1009,1043


In [8]:
# Modifiquem el diccionari per obtenir les victories totals, a casa, i fora de casa per lliga i temporada.
df_list = dict()

for s in seasons:
    matchs_dic = dict()
    matchs_dic["wins_home"]  = []
    matchs_dic["wins_away"]  = []
    matchs_dic["draws_home"] = []
    matchs_dic["draws_away"] = []
    matchs_dic["loses_home"] = []
    matchs_dic["loses_away"] = []
    for l in leagues:
        wins_home  = 0
        wins_away  = 0
        draws_home = 0
        draws_away = 0
        loses_home = 0
        loses_away = 0
        for t in list(match_season[s][l].values()):
            wins_home  += t["wins_home"]
            wins_away  += t["wins_away"]
            draws_home += t["draws_home"]
            draws_away += t["draws_away"]
            loses_home += t["loses_home"]
            loses_away += t["loses_away"]
            
        matchs_dic["wins_home"].append(wins_home)
        matchs_dic["wins_away"].append(wins_away)
        matchs_dic["draws_home"].append(draws_home)
        matchs_dic["draws_away"].append(draws_away)
        matchs_dic["loses_home"].append(loses_home)
        matchs_dic["loses_away"].append(loses_away)
    df = pd.DataFrame(matchs_dic, index = index_goals_tt[:11])
    df_list[s] = df

In [9]:
# Temporada 2008/2009.
df_list['2008/2009']

,wins_home,wins_away,draws_home,draws_away,loses_home,loses_away
Belgium Jupiler League,149,90,67,67,90,149
England Premier League,173,110,97,97,110,173
France Ligue 1,165,103,112,112,103,165
Germany 1. Bundesliga,147,85,74,74,85,147
Italy Serie A,192,93,95,95,93,192
Netherlands Eredivisie,147,83,76,76,83,147
Poland Ekstraklasa,119,62,59,59,62,119
Portugal Liga ZON Sagres,107,69,64,64,69,107
Scotland Premier League,96,73,59,59,73,96
Spain LIGA BBVA,184,113,83,83,113,184


In [10]:
# Temporada 2009/2010.
df_list['2009/2010']

,wins_home,wins_away,draws_home,draws_away,loses_home,loses_away
Belgium Jupiler League,97,62,51,51,62,97
England Premier League,193,91,96,96,91,193
France Ligue 1,179,104,97,97,104,179
Germany 1. Bundesliga,125,95,86,86,95,125
Italy Serie A,186,92,102,102,92,186
Netherlands Eredivisie,153,91,62,62,91,153
Poland Ekstraklasa,108,67,65,65,67,108
Portugal Liga ZON Sagres,104,68,68,68,68,104
Scotland Premier League,94,71,63,63,71,94
Spain LIGA BBVA,194,91,95,95,91,194


In [11]:
# Temporada 2010/2011.
df_list['2010/2011']

,wins_home,wins_away,draws_home,draws_away,loses_home,loses_away
Belgium Jupiler League,116,62,62,62,62,116
England Premier League,179,90,111,111,90,179
France Ligue 1,157,93,130,130,93,157
Germany 1. Bundesliga,141,102,63,63,102,141
Italy Serie A,179,104,97,97,104,179
Netherlands Eredivisie,160,75,71,71,75,160
Poland Ekstraklasa,124,56,60,60,56,124
Portugal Liga ZON Sagres,101,73,66,66,73,101
Scotland Premier League,94,85,49,49,85,94
Spain LIGA BBVA,197,104,79,79,104,197


In [12]:
# Temporada 2011/2012.
df_list['2011/2012']

,wins_home,wins_away,draws_home,draws_away,loses_home,loses_away
Belgium Jupiler League,119,56,65,65,56,119
England Premier League,171,116,93,93,116,171
France Ligue 1,179,93,108,108,93,179
Germany 1. Bundesliga,139,88,79,79,88,139
Italy Serie A,165,92,101,101,92,165
Netherlands Eredivisie,155,88,63,63,88,155
Poland Ekstraklasa,107,68,65,65,68,107
Portugal Liga ZON Sagres,115,72,53,53,72,115
Scotland Premier League,90,81,57,57,81,90
Spain LIGA BBVA,188,98,94,94,98,188


In [13]:
# Temporada 2012/2013.
df_list['2012/2013']

,wins_home,wins_away,draws_home,draws_away,loses_home,loses_away
Belgium Jupiler League,102,79,59,59,79,102
England Premier League,166,106,108,108,106,166
France Ligue 1,170,102,108,108,102,170
Germany 1. Bundesliga,130,98,78,78,98,130
Italy Serie A,177,107,96,96,107,177
Netherlands Eredivisie,137,91,78,78,91,137
Poland Ekstraklasa,97,80,63,63,80,97
Portugal Liga ZON Sagres,103,76,61,61,76,103
Scotland Premier League,89,66,73,73,66,89
Spain LIGA BBVA,189,107,84,84,107,189


In [14]:
# Temporada 2013/2014.
df_list['2013/2014']

,wins_home,wins_away,draws_home,draws_away,loses_home,loses_away
Belgium Jupiler League,6,4,2,2,4,6
England Premier League,179,123,78,78,123,179
France Ligue 1,168,104,108,108,104,168
Germany 1. Bundesliga,145,97,64,64,97,145
Italy Serie A,181,109,90,90,109,181
Netherlands Eredivisie,144,78,84,84,78,144
Poland Ekstraklasa,110,57,73,73,57,110
Portugal Liga ZON Sagres,108,72,60,60,72,108
Scotland Premier League,102,76,50,50,76,102
Spain LIGA BBVA,179,115,86,86,115,179


In [15]:
# Temporada 2014/2015.
df_list['2014/2015']

,wins_home,wins_away,draws_home,draws_away,loses_home,loses_away
Belgium Jupiler League,106,74,60,60,74,106
England Premier League,172,115,93,93,115,172
France Ligue 1,181,111,88,88,111,181
Germany 1. Bundesliga,145,79,82,82,79,145
Italy Serie A,152,107,120,120,107,152
Netherlands Eredivisie,138,95,73,73,95,138
Poland Ekstraklasa,114,60,66,66,60,114
Portugal Liga ZON Sagres,137,84,85,85,84,137
Scotland Premier League,102,82,44,44,82,102
Spain LIGA BBVA,171,118,91,91,118,171


In [16]:
# Temporada 2015/2016.
df_list['2015/2016']

,wins_home,wins_away,draws_home,draws_away,loses_home,loses_away
Belgium Jupiler League,115,66,59,59,66,115
England Premier League,157,116,107,107,116,157
France Ligue 1,160,112,108,108,112,160
Germany 1. Bundesliga,135,100,71,71,100,135
Italy Serie A,175,110,95,95,110,175
Netherlands Eredivisie,137,95,74,74,95,137
Poland Ekstraklasa,91,75,74,74,75,91
Portugal Liga ZON Sagres,133,97,76,76,97,133
Scotland Premier League,93,83,52,52,83,93
Spain LIGA BBVA,183,105,92,92,105,183


In [17]:
best_season  = dict()
worst_season = dict()

for s in seasons:
    season_stats = dict()
    for l in leagues:
        value_max = 0
        name_max = ""
        value_min = 999
        name_min = ""
        for t, name in zip(list(match_season[s][l].values()), list(match_season[s][l].keys())):
            value = t["wins"]*3 + t["draws"]
            if value > value_max:
                value_max = value
                name_max  = name
            if value < value_min:
                value_min = value
                name_min  = name
        
        if l in best_season.keys():
            if best_season[l]["points"] < value_max:
                best_season[l] = {"season": s, "team": name_max, "points": value_max}
        else:
            best_season[l] = {"season": s, "team": name_max, "points": value_max}
            
        if l in worst_season.keys():
            if worst_season[l]["points"] > value_min:
                worst_season[l] = {"season": s, "team": name_min, "points": value_min}
        else:
            worst_season[l] = {"season": s, "team": name_min, "points": value_min}

In [18]:
best_list = []
properties = ["wins_home", "wins_away", "draws_home", "draws_away", "loses_home", "loses_away",
              "scores_home", "scores_away", "scores_rec_home", "scores_rec_away"]
for k,v in best_season.items():
    team = match_season[v["season"]][k][v["team"]]
    
    data_team = {k: [
        team[properties[0]],
        team[properties[1]],
        team[properties[2]],
        team[properties[3]],
        team[properties[4]],
        team[properties[5]],
        team[properties[6]],
        team[properties[7]],
        team[properties[8]],
        team[properties[9]]
    ]}
    
    df = pd.DataFrame(data_team, index = properties)
    best_list.append(df)

In [19]:
best_list[0]

,Belgium Jupiler League
wins_home,14
wins_away,10
draws_home,1
draws_away,4
loses_home,2
loses_away,3
scores_home,48
scores_away,27
scores_rec_home,15
scores_rec_away,15


In [20]:
best_list[1]

,England Premier League
wins_home,16
wins_away,12
draws_home,2
draws_away,4
loses_home,1
loses_away,3
scores_home,43
scores_away,25
scores_rec_home,13
scores_rec_away,11


In [21]:
best_list[2]

,France Ligue 1
wins_home,15
wins_away,15
draws_home,3
draws_away,3
loses_home,1
loses_away,1
scores_home,59
scores_away,43
scores_rec_home,12
scores_rec_away,7


In [22]:
best_list[3]

,Germany 1. Bundesliga
wins_home,14
wins_away,15
draws_home,2
draws_away,2
loses_home,1
loses_away,0
scores_home,56
scores_away,42
scores_rec_home,11
scores_rec_away,7


In [23]:
best_list[4]

,Italy Serie A
wins_home,19
wins_away,14
draws_home,0
draws_away,3
loses_home,0
loses_away,2
scores_home,47
scores_away,33
scores_rec_home,9
scores_rec_away,14


In [24]:
best_list[5]

,Netherlands Eredivisie
wins_home,16
wins_away,13
draws_home,0
draws_away,1
loses_home,1
loses_away,3
scores_home,50
scores_away,42
scores_rec_home,12
scores_rec_away,19


In [25]:
best_list[6]

,Poland Ekstraklasa
wins_home,13
wins_away,6
draws_home,9
draws_away,6
loses_home,8
loses_away,18
scores_home,41
scores_away,24
scores_rec_home,28
scores_rec_away,57


In [26]:
best_list[7]

,Portugal Liga ZON Sagres
wins_home,15
wins_away,14
draws_home,0
draws_away,1
loses_home,2
loses_away,2
scores_home,52
scores_away,36
scores_rec_home,13
scores_rec_away,9


In [27]:
best_list[8]

,Scotland Premier League
wins_home,16
wins_away,15
draws_home,3
draws_away,3
loses_home,0
loses_away,1
scores_home,50
scores_away,52
scores_rec_home,10
scores_rec_away,15


In [28]:
best_list[9]

,Spain LIGA BBVA
wins_home,16
wins_away,16
draws_home,2
draws_away,2
loses_home,1
loses_away,1
scores_home,70
scores_away,51
scores_rec_home,19
scores_rec_away,13


In [29]:
best_list[10]

,Switzerland Super League
wins_home,14
wins_away,12
draws_home,2
draws_away,3
loses_home,2
loses_away,3
scores_home,44
scores_away,44
scores_rec_home,15
scores_rec_away,23


In [30]:
worst_list = []
for k,v in worst_season.items():
    team = match_season[v["season"]][k][v["team"]]
    
    data_team = {k: [
        team[properties[0]],
        team[properties[1]],
        team[properties[2]],
        team[properties[3]],
        team[properties[4]],
        team[properties[5]],
        team[properties[6]],
        team[properties[7]],
        team[properties[8]],
        team[properties[9]]
    ]}
    
    df = pd.DataFrame(data_team, index = properties)
    worst_list.append(df)

In [31]:
worst_list[0]

,Belgium Jupiler League
wins_home,1
wins_away,0
draws_home,0
draws_away,1
loses_home,2
loses_away,2
scores_home,4
scores_away,3
scores_rec_home,5
scores_rec_away,6


In [32]:
worst_list[1]

,England Premier League
wins_home,2
wins_away,1
draws_home,5
draws_away,3
loses_home,12
loses_away,15
scores_home,14
scores_away,13
scores_rec_home,35
scores_rec_away,41


In [33]:
worst_list[2]

,France Ligue 1
wins_home,1
wins_away,2
draws_home,7
draws_away,2
loses_home,11
loses_away,15
scores_home,13
scores_away,15
scores_rec_home,36
scores_rec_away,47


In [34]:
worst_list[3]

,Germany 1. Bundesliga
wins_home,0
wins_away,4
draws_home,4
draws_away,5
loses_home,13
loses_away,8
scores_home,10
scores_away,16
scores_rec_home,36
scores_rec_away,24


In [35]:
worst_list[4]

,Italy Serie A
wins_home,2
wins_away,2
draws_home,6
draws_away,2
loses_home,9
loses_away,13
scores_home,15
scores_away,9
scores_rec_home,22
scores_rec_away,34


In [36]:
worst_list[5]

,Netherlands Eredivisie
wins_home,4
wins_away,1
draws_home,0
draws_away,0
loses_home,13
loses_away,16
scores_home,18
scores_away,12
scores_rec_home,30
scores_rec_away,50


In [37]:
worst_list[6]

,Poland Ekstraklasa
wins_home,3
wins_away,1
draws_home,4
draws_away,6
loses_home,8
loses_away,8
scores_home,10
scores_away,10
scores_rec_home,20
scores_rec_away,21


In [38]:
worst_list[7]

,Portugal Liga ZON Sagres
wins_home,4
wins_away,1
draws_home,2
draws_away,2
loses_home,9
loses_away,12
scores_home,14
scores_away,11
scores_rec_home,29
scores_rec_away,27


In [39]:
worst_list[8]

,Scotland Premier League
wins_home,1
wins_away,4
draws_home,7
draws_away,3
loses_home,11
loses_away,12
scores_home,22
scores_away,18
scores_rec_home,44
scores_rec_away,38


In [40]:
worst_list[9]

,Spain LIGA BBVA
wins_home,1
wins_away,2
draws_home,6
draws_away,5
loses_home,12
loses_away,12
scores_home,12
scores_away,10
scores_rec_home,33
scores_rec_away,35


In [41]:
worst_list[10]

,Switzerland Super League
wins_home,4
wins_away,1
draws_home,5
draws_away,2
loses_home,9
loses_away,15
scores_home,19
scores_away,9
scores_rec_home,35
scores_rec_away,50
